# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.compat.v1.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
notebook_folder = os.path.abspath('')
train_csv_path = os.path.join(notebook_folder, 'Project_data\\train.csv')
val_csv_path = os.path.join(notebook_folder, 'Project_data\\val.csv')

# Read the content of train.csv and val.csv
train_doc = np.random.permutation(open(train_csv_path).readlines())
train_doc = list(train_doc)
val_doc = np.random.permutation(open(val_csv_path).readlines())
val_doc = list(val_doc)

print(train_csv_path)
print(val_csv_path)
print(len(train_doc), len(val_doc))

batch_size = 20

D:\UpGrad\assignments\GestureRecognitionNN\Project_data\train.csv
D:\UpGrad\assignments\GestureRecognitionNN\Project_data\val.csv
663 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
# def generator(source_path, folder_list, batch_size, fraction = 1.0):
#     print( 'Source path = ', source_path, '; batch size =', batch_size)
#     img_idx = [i for i in range(0, 30)]
#     dataset_size = int(len(folder_list) * fraction)
#     folder_list = folder_list[:dataset_size]
#     while True:
#         t = np.random.permutation(folder_list)
#         num_batches = len(folder_list) // batch_size  
#         for batch in range(num_batches): # we iterate over the number of batches
#             batch_data = np.zeros((batch_size,30,120,160,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
#             batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
#             for folder in range(batch_size): # iterate over the batch_size
#                 imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
#                 for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
#                     image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
#                     image = imresize(image, (120, 160)).astype(np.float32)
                    
#                     # Normalize and feed in the image
#                     batch_data[folder, idx, :, :, 0] = (image[:, :, 0] - image[:, :, 0].min()) / (image[:, :, 0].max() - image[:, :, 0].min())
#                     batch_data[folder, idx, :, :, 1] = (image[:, :, 1] - image[:, :, 1].min()) / (image[:, :, 1].max() - image[:, :, 1].min())
#                     batch_data[folder, idx, :, :, 2] = (image[:, :, 2] - image[:, :, 2].min()) / (image[:, :, 2].max() - image[:, :, 2].min())
                    
#                 batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
#             yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
#         # write the code for the remaining data points which are left after full batches
#         remaining_batch_size = len(folder_list) % batch_size
#         if remaining_batch_size != 0:
#             batch_data = np.zeros((remaining_batch_size, 30, 120, 160, 3))
#             batch_labels = np.zeros((remaining_batch_size, 5))
#             for folder in range(remaining_batch_size):
#                 # Similar processing as in the main batch loop
#                 imgs = os.listdir(os.path.join(source_path, t[folder + (num_batches * batch_size)].split(';')[0]))
#                 for idx, item in enumerate(img_idx):
#                     image = imread(os.path.join(source_path, t[folder + (num_batches * batch_size)].strip().split(';')[0], imgs[item])).astype(np.float32)
#                     image = imresize(image, (120, 160)).astype(np.float32)
#                     batch_data[folder, idx, :, :, 0] = (image[:, :, 0] - image[:, :, 0].min()) / (image[:, :, 0].max() - image[:, :, 0].min())
#                     batch_data[folder, idx, :, :, 1] = (image[:, :, 1] - image[:, :, 1].min()) / (image[:, :, 1].max() - image[:, :, 1].min())
#                     batch_data[folder, idx, :, :, 2] = (image[:, :, 2] - image[:, :, 2].min()) / (image[:, :, 2].max() - image[:, :, 2].min())

#                 batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
#             yield batch_data, batch_labels



In [4]:
# The fraction value is used to subset the data for training
def generator(source_path, folder_list, batch_size, fraction=1.0):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0, 30)]
    
    # Calculate the dataset size based on the specified fraction
    dataset_size = int(len(folder_list) * fraction)
    folder_list = folder_list[:dataset_size]
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size  
        
        for batch in range(num_batches): # iterate over the number of batches
            batch_data = np.zeros((batch_size, 30, 120, 160, 3))  # x, y, z, 3
            batch_labels = np.zeros((batch_size, 5))  # 5 gestures
            
            for folder in range(batch_size):  # iterate over the batch_size
                imgs = os.listdir(os.path.join(source_path, t[folder + (batch * batch_size)].split(';')[0]))
                
                for idx, item in enumerate(img_idx):  # iterate over the frames/images of a folder to read them in
                    image = imread(os.path.join(source_path, t[folder + (batch * batch_size)].strip().split(';')[0], imgs[item])).astype(np.float32)
                    
                    # Crop and resize the image
                    image = imresize(image, (120, 160)).astype(np.float32)
                    
                    # Normalize and feed in the image
                    batch_data[folder, idx, :, :, 0] = (image[:, :, 0] - image[:, :, 0].min()) / (image[:, :, 0].max() - image[:, :, 0].min())
                    batch_data[folder, idx, :, :, 1] = (image[:, :, 1] - image[:, :, 1].min()) / (image[:, :, 1].max() - image[:, :, 1].min())
                    batch_data[folder, idx, :, :, 2] = (image[:, :, 2] - image[:, :, 2].min()) / (image[:, :, 2].max() - image[:, :, 2].min())
                    
                batch_labels[folder, int(t[folder + (batch * batch_size)].strip().split(';')[2])] = 1
            
            yield batch_data, batch_labels  # yield the batch_data and the batch_labels

        # Handle remaining data points after full batches
        remaining_batch_size = len(folder_list) % batch_size
        if remaining_batch_size != 0:
            batch_data = np.zeros((remaining_batch_size, 30, 120, 160, 3))
            batch_labels = np.zeros((remaining_batch_size, 5))
            
            for folder in range(remaining_batch_size):
                imgs = os.listdir(os.path.join(source_path, t[folder + (num_batches * batch_size)].split(';')[0]))
                
                for idx, item in enumerate(img_idx):
                    image = imread(os.path.join(source_path, t[folder + (num_batches * batch_size)].strip().split(';')[0], imgs[item])).astype(np.float32)
                    image = imresize(image, (120, 160)).astype(np.float32)
                    
                    batch_data[folder, idx, :, :, 0] = (image[:, :, 0] - image[:, :, 0].min()) / (image[:, :, 0].max() - image[:, :, 0].min())
                    batch_data[folder, idx, :, :, 1] = (image[:, :, 1] - image[:, :, 1].min()) / (image[:, :, 1].max() - image[:, :, 1].min())
                    batch_data[folder, idx, :, :, 2] = (image[:, :, 2] - image[:, :, 2].min()) / (image[:, :, 2].max() - image[:, :, 2].min())

                batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
            
            yield batch_data, batch_labels

        # Shuffle the dataset for the next epoch
        folder_list = folder_list[num_batches * batch_size:] + folder_list[:num_batches * batch_size]


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = os.path.join(notebook_folder, 'Project_data\\train')
val_path = os.path.join(notebook_folder, 'Project_data\\val')
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

model = Sequential()

model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [10]:
optimiser = 'adam'
input_shape = (20,30,120,160,3)
model.build(input_shape)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (20, 30, 120, 160, 64)    5248      
                                                                 
 max_pooling3d (MaxPooling3  (20, 15, 60, 80, 64)      0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (20, 15, 60, 80, 128)     221312    
                                                                 
 max_pooling3d_1 (MaxPoolin  (20, 7, 30, 40, 128)      0         
 g3D)                                                            
                                                                 
 dropout (Dropout)           (20, 7, 30, 40, 128)      0         
                                                                 
 flatten (Flatten)           (20, 1075200)             0

In [ ]:
# training_fraction = 0.2
# small_train_generator = generator(train_path, train_doc, batch_size, fraction = training_fraction)
# small_val_generator = generator(val_path, val_doc, batch_size, fraction = training_fraction)

In [ ]:
# small_data_num_train_sequences = num_train_sequences*training_fraction
# small_data_num_val_sequences = num_val_sequences*training_fraction
# if (small_data_num_train_sequences%batch_size) == 0:
#     small_data_steps_per_epoch = int(small_data_num_train_sequences/batch_size)
# else:
#     small_data_steps_per_epoch = (small_data_num_train_sequences//batch_size) + 1

# if (num_val_sequences%batch_size) == 0:
#     small_data_validation_steps = int(small_data_num_val_sequences/batch_size)
# else:
#     small_data_validation_steps = (small_data_num_val_sequences//batch_size) + 1

In [ ]:
# model_name = 'model_init_small_data' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
# if not os.path.exists(model_name):
#     os.mkdir(model_name)
        
# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
# callbacks_list = [checkpoint, LR]

In [ ]:
# model.fit_generator(small_train_generator, steps_per_epoch=small_data_steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=small_val_generator, 
#                     validation_steps=small_data_validation_steps, class_weight=None, workers=1, initial_epoch=0)

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
%time model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, callbacks=callbacks_list, validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<timed eval>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
C:\Users\abhir\AppData\Local\Temp\ipykernel_26996\3699764615.py:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(os.path.join(source_path, t[folder + (batch * batch_size)].strip().split(';')[0], imgs[item])).astype(np.float32)


Source path =  D:\UpGrad\assignments\GestureRecognitionNN\Project_data\train ; batch size = 20
Epoch 1/20
31/34 [==========================>...] - ETA: 2:58 - loss: 9.9996 - categorical_accuracy: 0.2242 

C:\Users\abhir\AppData\Local\Temp\ipykernel_26996\3699764615.py:45: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(os.path.join(source_path, t[folder + (num_batches * batch_size)].strip().split(';')[0], imgs[item])).astype(np.float32)


34/34 [==============================] - ETA: 0s - loss: 9.4543 - categorical_accuracy: 0.2262 Source path =  D:\UpGrad\assignments\GestureRecognitionNN\Project_data\val ; batch size = 20

Epoch 1: saving model to model_init_2023-12-1308_27_31.050287\model-00001-9.45426-0.22624-1.59259-0.35000.h5


C:\Users\abhir\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 2013s 59s/step - loss: 9.4543 - categorical_accuracy: 0.2262 - val_loss: 1.5926 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.5480 - categorical_accuracy: 0.3560 
Epoch 2: saving model to model_init_2023-12-1308_27_31.050287\model-00002-1.54802-0.35596-1.33945-0.28000.h5
34/34 [==============================] - 1935s 57s/step - loss: 1.5480 - categorical_accuracy: 0.3560 - val_loss: 1.3394 - val_categorical_accuracy: 0.2800 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.2553 - categorical_accuracy: 0.5158 
Epoch 3: saving model to model_init_2023-12-1308_27_31.050287\model-00003-1.25525-0.51584-1.29438-0.47000.h5
34/34 [==============================] - 1931s 57s/step - loss: 1.2553 - categorical_accuracy: 0.5158 - val_loss: 1.2944 - val_categorical_accuracy: 0.4700 - lr: 0.0010
Epoch 4/20
34/34 [==============================] - ETA: 0s -